In [44]:
import pandas as pd
import numpy as np
from my_api import *
import matplotlib.pyplot as plt

In [45]:
id = pd.MultiIndex.from_product([['fro', 2], [100, 300, 500]])
df = pd.DataFrame(index = id)

In [46]:
T = 300
rela = bool(0)
cov_str = 'Cai2011Adaptive_Model2_my'
# cov_str = 'Cai2011Adaptive_Model1'

In [47]:
simu_str = 'lx_band'
data_path = 'data_2023-4-1/'
for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0)
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for eta in [0.5, 0.8, 1]:
            cov_params = [cov_str]
            simu_params = [simu_str, eta]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            data = np.loadtxt(file_name)

            col = str(simu_params)
            if col not in df.columns:
                df[col] = None
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"

In [48]:
simu_str = 'LSRthreshold'
data_path = 'data_LSRthreshold_cv50_4-9/'
for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0)
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for tau in [0.2]:
            # for prob in [0.99, 0.9, 1]:
            #     for qrob in [0, 0.01, 0.1]:
            for prob in [1, 0.99]:
                for qrob in [0, 0.01]:
                    cov_params = [cov_str]
                    simu_params = [simu_str, tau, prob, qrob, 'brute', 10]
                    file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
                    file_name = data_path + file_name
                    
                    try:
                        data = np.loadtxt(file_name)
                    except FileNotFoundError:
                        continue

                    col = str(simu_params)
                    if col not in df.columns:
                        df[col] = None
                    if not rela:
                        data *= normS # absolute error
                    df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"

In [49]:
simu_str = '' # to be determined
data_path1 = 'data_OtherMethods_4-5/'
data_path2 = 'data_other_threshold_cv50_4-9/'

for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0)
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)

        # from Cai2011_other import Other_Methods
        for simu_str in ['Sample', 'Soft Threshold', 'Hard Threshold', 'Linear Shrink', 'Nonlinear Shrink']:
            data_path = data_path2 if 'Threshold' in simu_str else data_path1
            print(data_path, simu_str)
 
            cov_params = [cov_str]
            simu_params = [simu_str]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            try:
                data = np.loadtxt(file_name)
            except FileNotFoundError:
                continue

            col = str(*simu_params)
            if col not in df.columns:
                df[col] = None
            
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"

In [50]:
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100       7.68(0.38)       4.57(0.26)     4.17(0.20)   
    300      21.03(0.34)      15.11(0.33)    13.77(0.33)   
    500      35.80(0.41)      27.26(0.44)    25.69(0.43)   
2   100       2.54(0.14)       1.48(0.14)     1.33(0.13)   
    300       4.02(0.14)       3.10(0.16)     3.08(0.18)   
    500       5.79(0.17)       4.21(0.15)     4.16(0.14)   

        ['LSRthreshold', 0.2, 1, 0, 'brute', 10]        Sample Soft Threshold  \
fro 100                              12.20(0.76)   19.71(0.31)    12.71(6.02)   
    300                              47.59(3.90)   77.44(0.47)   47.64(19.91)   
    500                              99.15(5.59)  147.70(0.70)   64.94(14.90)   
2   100                               3.71(0.36)    5.90(0.34)     3.65(1.86)   
    300                               8.15(0.99)   14.41(0.47)     8.98(3.71)   
    500                              15.75(0.81)   23.31(0.53)     9.88(1.57)   

        Hard Threshold Linear Shrink Nonlinear Shrink  
fro 100    16.39(5.23)   10.49(0.06)       9.15(0.12)  
    300   44.55(22.66)   29.47(0.07)             None  
    500    51.06(1.12)   51.83(0.08)             None  
2   100     5.35(1.06)    2.77(0.05)       2.16(0.08)  
    300     8.38(4.51)    7.20(0.16)             None  
    500     8.04(0.99)   11.48(0.19)             None

In [51]:
new_cols = []
for col in df.columns:
    try:
        col = eval(col)
    except Exception:
        pass

    if type(col) == str:
        new_col = col
    elif 'lx_band' in col:
        eta = col[1]
        new_col = fr'Augmented Band ($\eta={eta}$)'
    elif 'LSRthreshold' in col:
        t, p, q = col[1:-2]
        new_col = fr'Augmented Threshold ($\tau={t}, p={p}, q={q}$)' # The r in fr, means raw string. Otherwise I should write \\tau instead of \tau.
    new_cols.append(new_col)
df.columns = new_cols

In [52]:
buf = 'latex/' + cov_str + '.tex'
caption = cov_str

with pd.option_context("max_colwidth", 1000, 'display.width', 50):
    df.to_latex(buf = buf, 
                index = True, na_rep = ' ', sparsify = True, 
             column_format = 'll|' + 'p{2cm}' * df.shape[1], 
             longtable = False, 
             escape = False, encoding = 'utf-8', 
             caption = caption, label = "my label", 
             position = 'htbp')
# escape参数，防止字符被转换使得我设的数学模式被破坏

C:\Users\lx\AppData\Local\Temp\ipykernel_10172\4238430270.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(buf = buf,


In [53]:
pd.get_option('display.width')

80

In [54]:
lines = []

with open(buf, 'r') as fp:
    lines = fp.readlines()

s1 = r'\resizebox{\linewidth}{!}{' + '\n'
s2 = r'}' + '\n'
lines.insert(4, s1)
lines.insert(-1, s2)

with open(buf[:-4] + '_processed' + '.tex', 'w') as fp:
    fp.writelines(lines)
        # if not write_flag:
        #     if r'begin{tabular}' in line:
